In [ ]:
import datetime as dt
import pandas as pd
import yfinance as yf
# stbt is used only here, so it's not in requirements.txt
from stbt.download_ohlc.cryptocurrency import get_ohlc_cryptocompare, correct_ohlc_df
import lib.helpers as hlp

In [ ]:
historical_df = hlp.get_historical_prices_df()

In [ ]:
# historical_df['Tailed_Bitcoin'] = historical_df['Bitcoin']
# historical_df.at[dt.datetime(2017, 1, 1), 'Tailed_Bitcoin'] = 
# historical_df.sort_index(inplace=True)
# historical_df = historical_df.iloc[1:]
historical_df.at[dt.datetime(2021, 11, 1), 'Tailed_Bitcoin'] = 0
print(historical_df.tail(1))

In [ ]:
historical_df = historical_df.reindex(sorted(historical_df.columns), axis=1)
print(historical_df.columns)

In [ ]:
historical_df.to_csv(f'{hlp.INPUT_DATA_DIR}/historical_source.csv')

In [ ]:
instruments_list = [
    {'name': 'Bitcoin', 'ticker': 'BTC/USDT'},
    {'name': 'Litecoin', 'ticker': 'LTC/USDT'},
    {'name': 'XRP', 'ticker': 'XRP/USDT'},
    {'name': 'Dogecoin', 'ticker': 'DOGE/USDT'},
    {'name': 'Monero', 'ticker': 'XMR/USDT'},
    {'name': 'Stellar', 'ticker': 'XLM/USDT'},
    {'name': 'NEM', 'ticker': 'XEM/USDT'},
    {'name': 'Ethereum', 'ticker': 'ETH/USDT'},
    {'name': 'IOTA', 'ticker': 'MIOTA/USDT'},
    {'name': 'EOS', 'ticker': 'EOS/USDT'},
    {'name': 'Binance Coin', 'ticker': 'BNB/USDT'},
    {'name': 'TRON', 'ticker': 'TRX/USDT'},
    {'name': 'Chainlink', 'ticker': 'LINK/USDT'},
    {'name': 'Cardano', 'ticker': 'ADA/USDT'},
    {'name': 'USD Coin', 'ticker': 'USDC/USDT'},
    {'name': 'Crypto.com Coin', 'ticker': 'CRO/USDT'},
    {'name': 'Cosmos', 'ticker': 'ATOM/USDT'},
    {'name': 'Solana', 'ticker': 'SOL/USDT'},
    {'name': 'Polkadot', 'ticker': 'DOT/USDT'},
    {'name': 'Uniswap', 'ticker': 'UNI/USDT'},
    {'name': 'Aave', 'ticker': 'AAVE/USDT'},
]

In [ ]:
# use stbt specifically for downloadi
start_date = dt.datetime(2015, 3, 1, 0, 0, 0)
end_date = dt.datetime(2021, 11, 1, 0, 0, 0)


total_df = pd.DataFrame()

for instruments_dict in instruments_list:
    first_ticker, second_ticker = instruments_dict['ticker'].split('/')
    print(f'Processing {instruments_dict}')
    ohlc_df = get_ohlc_cryptocompare(first_ticker, second_ticker, start_date,
                                     end_date=end_date, interval_key='day')
    ohlc_df = correct_ohlc_df(ohlc_df)
    ohlc_df = ohlc_df[['Close']]
    ohlc_df.rename(columns={'Close': instruments_dict['name']}, inplace=True)
    if total_df.empty:
        total_df = ohlc_df
    else:
        total_df = total_df.merge(ohlc_df, how='left', on='Date')
    
total_df.index.names = ['date']


In [ ]:

print(total_df)

In [ ]:
total_filtered_df = total_df[total_df.index > dt.datetime(2017, 1, 1)]
total_filtered_df = total_filtered_df.reindex(sorted(total_filtered_df.columns), axis=1)
total_filtered_df.to_csv(f'{hlp.INPUT_DATA_DIR}/historical_source.csv')

In [ ]:
tickers_list = ['TSLA', 'SPY', 'EURUSD=x', 'USDTRY=X', 'MSCI', 'MSFT', '^N225', 'GC=F', 'BZ=F']

In [ ]:
for ticker in tickers_list:
    ohlc_df = yf.Ticker(ticker).history(period='5y')
    ohlc_df = ohlc_df[['Close']]
    ohlc_df.index.names = ['date']
    ohlc_df.rename(columns={'Close': ticker}, inplace=True)
    
    total_df = total_df.merge(ohlc_df, how='left', on='date')
    